In [1]:
import os
import pandas
import psycopg2

region = os.getenv('AWS_REGION')


df = pandas.read_parquet('../data/bger-2024-3-text.parquet')


In [2]:

test_cases = [
  {
    "title": "Litige transfrontalier en matière de protection des données",
    "content": "Une entreprise suisse basée à Zurich a transféré des données personnelles de ses clients vers un prestataire basé en Floride. Un client suisse s'est plaint que ses données aient été utilisées à des fins de marketing sans consentement explicite. L’affaire soulève la question de la conformité au droit suisse sur la protection des données et au principe du transfert international."
  },
  {
    "title": "Discrimination dans un marché réglementé",
    "content": "Un ingénieur software demandeur d'emploi de nationalité allemande affirme que sa candidature a été rejetée par une entreprise semi-publique suisse en raison de sa nationalité. Il invoque une discrimination contraire à l’accord sur la libre circulation des personnes et aux dispositions suisses sur l’égalité de traitement."
  },
  {
    "title": "Droit des brevets et concurrence déloyale",
    "content": "Une PME suisse accuse une entreprise concurrente d’avoir copié un procédé breveté dans le domaine de la chimie verte, utilisé dans toute la Suisse. Elle saisit le Tribunal fédéral des brevets pour violation de brevet, demandant l’interdiction de vente et des dommages-intérêts."
  },
  {
    "title": "Responsabilité de l’État en matière de cybersécurité",
    "content": "Un citoyen suisse voit ses données fiscales compromises après une cyberattaque sur un portail cantonal de déclaration en ligne. Il intente une action contre la Confédération, affirmant une responsabilité subsidiaire pour défaut de surveillance ou standards insuffisants de cybersécurité imposés aux cantons."
  },
  {
    "title": "Recours contre une décision de l’OFSP",
    "content": "Un thérapeute en médecine complémentaire attaque une décision de l’Office fédéral de la santé publique (OFSP) lui refusant l'autorisation de facturer via l’assurance de base (LAMal). Il estime que la décision viole son droit de pratiquer et demande au Tribunal administratif fédéral d’annuler la décision."
  },
  {
    "title": "Délit d’initié et marchés financiers",
    "content": "Un cadre supérieur d’une banque zurichoise est poursuivi pour avoir vendu des actions la veille d’une annonce de fusion confidentielle. La FINMA a transmis le dossier au Ministère public de la Confédération pour soupçon de délit d’initié, en application de la Loi sur les bourses (LBVM)."
  },
  {
    "title": "Asile et détention administrative",
    "content": "Un ressortissant érythréen, débouté de sa demande d’asile, fait recours contre sa mise en détention en vue du renvoi, décidée par le SEM. Il soutient que son renvoi est impossible et que sa détention viole l’article 5 CEDH et la jurisprudence fédérale sur la proportionnalité."
  },
  {
    "title": "Affaire douanière et taxation à l'importation",
    "content": "Une société suisse importe des pièces électroniques de Chine via un entrepôt sous douane à Bâle. L’Administration fédérale des douanes lui inflige une amende pour fausse déclaration de valeur. L’entreprise fait recours en affirmant avoir suivi les instructions d’un transitaire autorisé."
  },
  {
    "title": "Liberté religieuse et service civil",
    "content": "Un objecteur de conscience témoigne que le refus de son admission au service civil, malgré ses convictions religieuses pacifistes, viole sa liberté de conscience garantie par la Constitution fédérale. L’affaire est portée devant le Tribunal fédéral."
  },
  {
    "title": "Litige en matière d’accès à l’information",
    "content": "Un journaliste demande à l’OFEV l’accès à un rapport interne sur des risques environnementaux liés à une entreprise de recyclage. Le département refuse en invoquant des raisons de confidentialité industrielle. Le journaliste saisit le Préposé fédéral à la transparence puis fait recours devant le Tribunal administratif fédéral."
  },
  {
    "title": "Agissements potentiellement illégaux de la société TVT",
    "content": "Un avis, intitulé « Avis de travaux », de la société TVT a été affiché à l'entrée d'un immeuble situé dans la commune d'Ecublens, canton de Vaud. Un résident de cet immeuble s'inquiète de la véracité de cet avis, relevant que ni les locataires, ni la gérance, ni les propriétaires n'ont commandé de travaux. L'avis prétend également que « L'installation est obligatoire et gratuite », une affirmation pour laquelle le résident ne perçoit aucune base légale. Au-delà d'une simple pratique commerciale agressive, le résident craint que la société TVT ne se prévale de l'autorité communale pour s'introduire par la ruse dans les propriétés privées et se demande si de telles actions sont punies pénalement en vertu du droit suisse."
  },
  {
    "title": "Remboursement d'un voyage suite à l'annulation du vol par la compagnie aérienne",
    "content": "Nous avons réservé via HotelPlan un séjour à Madère qui avait planifié un vol avec Swiss. Apparemment le vol a été annulé et les alternatives proposées par HotelPlan ne nous conviennent pas du tout. La société HotelPlan doit-elle nous rembourser ?"
  }
]


In [3]:
from pathlib import Path
import onnxruntime as ort
from transformers import AutoTokenizer

# Load tokenizer
onnx_path = Path("../data/onnx_model").resolve()
tokenizer = AutoTokenizer.from_pretrained(str(onnx_path))

# Load ONNX model directly
session = ort.InferenceSession(str(onnx_path / "model.onnx"))

# Prepare inputs
case_number = -1
sentence = test_cases[case_number]["content"]
inputs = tokenizer(sentence, return_tensors="np")

# Rename input keys if needed (depends on your export)
onnx_inputs = {
    "input_ids": inputs["input_ids"],
    "attention_mask": inputs["attention_mask"]
}

# Run inference
outputs = session.run(None, onnx_inputs)[0]
sentence_embedding = outputs[0]

# Use embedding for search
conn = psycopg2.connect(
    host=os.getenv("AWS_AURORA_READER_ENDPOINT"),
    port=5432,
    dbname="fedcourtdecisions",
    user=os.getenv("AWS_AURORA_VECTORS_USERNAME"),
    password=os.getenv("AWS_AURORA_VECTORS_PASSWORD")
)
# Convert embedding in PostgreSQL vector format
pg_vector = "[" + ",".join(f"{x:.6f}" for x in sentence_embedding.tolist()) + "]"

# pgvector request
with conn.cursor() as cur:
    cur.execute(f"""
        SET ivfflat.probes = 20;
        SELECT doc_id
        FROM embeddings
        ORDER BY embedding <-> %s::vector
        LIMIT 10;
    """, (pg_vector, ))

    hit_ids = {row[0] for row in cur.fetchall()}
    print("Matched doc_ids:", hit_ids)

conn.close()

relevant_docs = [
    {"doc": doc, "ref": doc_ref}
    for doc, doc_ref in zip(
        df.loc[df["docref"].isin(hit_ids), "text"],
        df.loc[df["docref"].isin(hit_ids), "docref"]
    )
]
relevant_docs

OperationalError: connection to server at "prod-aurora-pgvector.cluster-ro-ch4imgw8kj3d.us-east-1.rds.amazonaws.com" (54.84.75.228), port 5432 failed: Connection timed out
	Is the server running on that host and accepting TCP/IP connections?


In [8]:
import openai
import os

api_key = os.environ["OPENAI_API_KEY"]

oai_client = openai.OpenAI(api_key=api_key)
# Set the OpenAI API key
question = f"Fournis un avis juridique en te fondant sur les documents joints concernant le cas suivant : {test_cases[case_number]['content']}"

# Combine the documents into a single prompt
context = "\n\n".join([f"Décision {doc['ref']}:\n{doc['doc']}" for doc in relevant_docs])

response = oai_client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a legal assistant who answers based only on provided documents."},
        {"role": "user", "content": f"{context}\n\nQuestion: {question}"}
    ],
    temperature=0
)

print(response.choices[0].message.content)


Sur la base des décisions fédérales fournies, voici un avis juridique concernant la situation d’un ingénieur software allemand demandeur d’emploi en Suisse, qui allègue une discrimination en raison de sa nationalité lors du recrutement par une entreprise semi-publique suisse.

1. **Cadre juridique applicable**

- L’Accord sur la libre circulation des personnes (ALCP) entre la Suisse et l’Union européenne interdit toute discrimination fondée sur la nationalité dans le domaine de l’emploi (cf. Décision 9C_259/2016, consid. 5.4.2.2).
- L’article 8 al. 2 de la Constitution fédérale suisse garantit également l’interdiction de la discrimination, notamment fondée sur la nationalité (cf. Décision 2D_34/2020, consid. 2.1).
- Le Tribunal fédéral applique ces principes en exigeant que les situations comparables soient traitées de manière égale, et que toute différence de traitement fondée sur la nationalité soit justifiée par un motif objectif légitime, nécessaire et proportionné (cf. Décision 2D

In [ ]:
text = "{\"analyses\": [{\"input_sentence\": \"Nous avons r\\u00e9serv\\u00e9 via HotelPlan un s\\u00e9jour \\u00e0 Mad\\u00e8re qui avait planifi\\u00e9 un vol avec Swiss. Apparemment le vol a \\u00e9t\\u00e9 annul\\u00e9 et les alternatives propos\\u00e9es par HotelPlan ne nous conviennent pas du tout. La soci\\u00e9t\\u00e9 HotelPlan doit-elle nous rembourser ?\", \"documents\": [{\"docref\": \"1C_442/2008\", \"url\": \"https://www.bger.ch/ext/eurospider/live/it/php/aza/http/index.php?lang=it&type=highlight_simple_query&page=55&from_date=13.06.2009&to_date=10.08.2009&sort=relevance&insertion_date=&top_subcollection_aza=all&query_words=&rank=545&azaclir=aza&highlight_docid=aza%3A%2F%2F09-07-2009-1C_442-2008&number_of_ranks=1132\"}, {\"docref\": \"1C_58/2010\", \"url\": \"https://www.bger.ch/ext/eurospider/live/it/php/aza/http/index.php?lang=it&type=highlight_simple_query&page=65&from_date=01.12.2010&to_date=29.01.2011&sort=relevance&insertion_date=&top_subcollection_aza=all&query_words=&rank=648&azaclir=aza&highlight_docid=aza%3A%2F%2F22-12-2010-1C_58-2010&number_of_ranks=1133\"}, {\"docref\": \"2C_888/2015\", \"url\": \"https://www.bger.ch/ext/eurospider/live/it/php/aza/http/index.php?lang=it&type=highlight_simple_query&page=64&from_date=21.04.2016&to_date=19.06.2016&sort=relevance&insertion_date=&top_subcollection_aza=all&query_words=&rank=638&azaclir=aza&highlight_docid=aza%3A%2F%2F23-05-2016-2C_888-2015&number_of_ranks=1281\"}, {\"docref\": \"2C_402/2008\", \"url\": \"https://www.bger.ch/ext/eurospider/live/it/php/aza/http/index.php?lang=it&type=highlight_simple_query&page=93&from_date=20.06.2008&to_date=18.08.2008&sort=relevance&insertion_date=&top_subcollection_aza=all&query_words=&rank=922&azaclir=aza&highlight_docid=aza%3A%2F%2F27-06-2008-2C_402-2008&number_of_ranks=1074\"}], \"analysis\": \"Les documents fournis concernent principalement des d\\u00e9cisions du Tribunal f\\u00e9d\\u00e9ral suisse relatives au droit a\\u00e9rien, \\u00e0 la planification a\\u00e9roportuaire, \\u00e0 la gestion des vols et \\u00e0 la d\\u00e9tention en vue du renvoi, mais ne traitent pas directement des droits des passagers a\\u00e9riens en cas d'annulation de vol ni des obligations des agences de voyage telles qu'HotelPlan.\\n\\nCependant, en se fondant sur les principes g\\u00e9n\\u00e9raux du droit suisse et les \\u00e9l\\u00e9ments tir\\u00e9s des d\\u00e9cisions relatives au droit a\\u00e9rien (notamment la d\\u00e9cision 2C_888/2015 qui traite de la concession de ligne a\\u00e9rienne et des relations entre compagnies a\\u00e9riennes), on peut d\\u00e9duire les points suivants :\\n\\n1. **Responsabilit\\u00e9 d'HotelPlan en cas d'annulation de vol**  \\n   HotelPlan, en tant qu'agence de voyage, est tenue de fournir les prestations contractuelles convenues, incluant le transport a\\u00e9rien r\\u00e9serv\\u00e9. Si le vol Swiss initialement pr\\u00e9vu est annul\\u00e9, HotelPlan doit proposer une alternative raisonnable ou proc\\u00e9der au remboursement du voyage, conform\\u00e9ment au contrat de voyage et aux dispositions l\\u00e9gales applicables (Code des obligations suisse, art. 363 ss. sur le contrat de voyage).\\n\\n2. **Obligation de proposer des alternatives ad\\u00e9quates**  \\n   Les alternatives propos\\u00e9es doivent \\u00eatre acceptables pour le client. Si les solutions offertes par HotelPlan ne conviennent pas du tout, l'agence doit envisager d'autres options ou permettre au client de se r\\u00e9tracter du contrat et d'obtenir un remboursement.\\n\\n3. **Droit au remboursement**  \\n   En cas d'annulation de vol sans alternative acceptable, le client a droit au remboursement int\\u00e9gral des prestations non fournies. Ceci est conforme au droit suisse et aux principes g\\u00e9n\\u00e9raux du contrat de voyage.\\n\\n4. **Absence de dispositions sp\\u00e9cifiques dans les documents fournis**  \\n   Les d\\u00e9cisions jointes ne contiennent pas de r\\u00e8gles sp\\u00e9cifiques sur les droits des passagers en cas d'annulation de vol ni sur les obligations des agences de voyage dans ce contexte. Elles concernent plut\\u00f4t la planification a\\u00e9roportuaire, la gestion des infrastructures, les concessions de lignes a\\u00e9riennes et des questions de d\\u00e9tention administrative.\\n\\n**Conclusion juridique** :  \\nSur la base des principes g\\u00e9n\\u00e9raux du droit suisse et en l'absence de dispositions contraires dans les documents fournis, la soci\\u00e9t\\u00e9 HotelPlan est tenue de rembourser le client si le vol Swiss r\\u00e9serv\\u00e9 a \\u00e9t\\u00e9 annul\\u00e9 et que les alternatives propos\\u00e9es ne sont pas acceptables. Le client peut exiger le remboursement int\\u00e9gral des prestations non fournies. Il est conseill\\u00e9 de v\\u00e9rifier les conditions g\\u00e9n\\u00e9rales de vente d'HotelPlan et, le cas \\u00e9ch\\u00e9ant, de faire valoir ses droits aupr\\u00e8s de l'agence, voire par voie judiciaire si n\\u00e9cessaire.\"}]}"
import json
x = json.loads(text)
x

{'analyses': [{'input_sentence': 'Nous avons réservé via HotelPlan un séjour à Madère qui avait planifié un vol avec Swiss. Apparemment le vol a été annulé et les alternatives proposées par HotelPlan ne nous conviennent pas du tout. La société HotelPlan doit-elle nous rembourser ?',
   'documents': [{'docref': '1C_442/2008',
     'url': 'https://www.bger.ch/ext/eurospider/live/it/php/aza/http/index.php?lang=it&type=highlight_simple_query&page=55&from_date=13.06.2009&to_date=10.08.2009&sort=relevance&insertion_date=&top_subcollection_aza=all&query_words=&rank=545&azaclir=aza&highlight_docid=aza%3A%2F%2F09-07-2009-1C_442-2008&number_of_ranks=1132'},
    {'docref': '1C_58/2010',
     'url': 'https://www.bger.ch/ext/eurospider/live/it/php/aza/http/index.php?lang=it&type=highlight_simple_query&page=65&from_date=01.12.2010&to_date=29.01.2011&sort=relevance&insertion_date=&top_subcollection_aza=all&query_words=&rank=648&azaclir=aza&highlight_docid=aza%3A%2F%2F22-12-2010-1C_58-2010&number_of_r